# Job Search Agent Tools Testing

This notebook tests all 4 tools available for the Job Search Agent:
1. **search_linkedin_jobs** - LinkedIn job search with filters
2. **parse_cv_content** - CV/Resume parsing from text or PDF
3. **extract_pdf_text** - PDF text extraction
4. **search_jobs_from_cv** - Smart CV-based job search

## Setup

In [ ]:
import sys
import os

# Add backend directory to Python path
backend_path = os.path.join(os.path.dirname(os.getcwd()), 'backend')
if backend_path not in sys.path:
    sys.path.append(backend_path)

print(f"Backend path: {backend_path}")
print(f"Current working directory: {os.getcwd()}")

# Import model configuration from config
from config import MODEL_NAME
print(f"Using model: {MODEL_NAME}")

In [ ]:
# Import Job Search Agent
try:
    from job_search_agent import create_linkedin_job_agent
    
    print("✅ Job Search Agent imported successfully!")
    print(f"create_linkedin_job_agent: {create_linkedin_job_agent}")
    
    # Create the agent
    agent = create_linkedin_job_agent()
    print("✅ Agent created successfully!")
    print(f"Agent: {agent}")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Make sure you're running this from the tester directory")
except Exception as e:
    print(f"❌ Error creating agent: {e}")


In [ ]:
# Load environment variables
from dotenv import load_dotenv
import json

load_dotenv()

# Check if API keys are available
api_key = os.getenv('CUSTOM_SEARCH_API_KEY')
search_engine_id = os.getenv('GOOGLE_SEARCH_ENGINE_ID')

if api_key and search_engine_id:
    print("✅ API credentials found")
else:
    print("⚠️ API credentials not found. Some tests may fail.")


## Test Data Setup

Let's create some sample CV content for testing

In [ ]:
# Sample CV content for testing
sample_cv_text = """
John Doe
Senior Software Engineer

Contact Information:
Email: john.doe@email.com
Phone: +1-555-123-4567
LinkedIn: linkedin.com/in/johndoe
Location: San Francisco, CA

Professional Summary:
Experienced software engineer with 7+ years of experience in full-stack development, 
specializing in Python, JavaScript, and cloud technologies. Proven track record of 
leading development teams and delivering scalable web applications.

Work Experience:

Senior Software Engineer | TechCorp Inc. | 2021 - Present
San Francisco, CA
- Led a team of 5 developers in building microservices architecture
- Implemented CI/CD pipelines using Docker and Kubernetes
- Developed RESTful APIs using Python Django and FastAPI
- Collaborated with product team to deliver features for 1M+ users

Software Engineer | StartupXYZ | 2019 - 2021
Remote
- Built responsive web applications using React and Node.js
- Designed and implemented PostgreSQL database schemas
- Integrated third-party APIs and payment systems
- Mentored junior developers and conducted code reviews

Junior Developer | WebSolutions Ltd. | 2017 - 2019
New York, NY
- Developed front-end components using HTML, CSS, and JavaScript
- Maintained legacy PHP applications
- Participated in agile development processes

Education:
Bachelor of Science in Computer Science
University of California, Berkeley | 2013 - 2017
GPA: 3.7/4.0

Technical Skills:
- Programming Languages: Python, JavaScript, TypeScript, Java, PHP
- Frameworks: Django, FastAPI, React, Node.js, Express
- Databases: PostgreSQL, MySQL, MongoDB, Redis
- Cloud: AWS, Docker, Kubernetes, Terraform
- Tools: Git, Jenkins, JIRA, VS Code

Certifications:
- AWS Certified Solutions Architect (2022)
- Certified Kubernetes Administrator (2021)

Preferred Work Arrangement: Remote or Hybrid
Expected Salary: $120,000 - $150,000
"""

print("Sample CV text prepared:")
print(f"Length: {len(sample_cv_text)} characters")
print(f"Lines: {len(sample_cv_text.split('\\n'))} lines")


## 1. Test Agent CV Parsing Capability

Testing the agent's ability to parse CVs when user provides CV content

In [ ]:
print("🧪 Testing Agent CV Parsing Capability")
print("=" * 50)

try:
    # Test 1: User asks agent to analyze their CV
    user_query_1 = f"""Please analyze my CV and tell me about my background:

{sample_cv_text}

Can you extract my key information including experience, skills, and career level?"""
    
    print("Test 1: CV Analysis Request")
    print("Query: 'Please analyze my CV and tell me about my background...'")
    print("\n🤖 Agent Response:")
    
    response_1 = agent.invoke({"input": user_query_1})
    print(response_1.get("output", "No response"))
    
except Exception as e:
    print(f"❌ Exception during agent CV analysis: {e}")

print("\n" + "="*60)


## 2. Test Agent Job Search Capability

Testing the agent's ability to search for jobs based on user criteria

In [ ]:
print("🧪 Testing Agent Job Search Capability")
print("=" * 50)

try:
    # Test 1: Basic job search request
    user_query_2 = """I'm looking for Python Developer jobs in San Francisco, CA. 
    Can you help me find some opportunities? Please show me a few relevant positions with details about the companies, requirements, and application links."""
    
    print("Test 1: Basic Job Search Request")
    print("Query: 'I'm looking for Python Developer jobs in San Francisco, CA...'")
    print("\n🤖 Agent Response:")
    
    response_2 = agent.invoke({"input": user_query_2})
    print(response_2.get("output", "No response"))
    
except Exception as e:
    print(f"❌ Exception during agent job search: {e}")

print("\n" + "="*60)

try:
    # Test 2: Advanced job search with filters
    user_query_3 = """I want to find remote Senior Software Engineer positions. 
    I'm specifically interested in full-time roles at technology companies. 
    Can you search for recent job postings from the past week? Please find 3-4 relevant opportunities."""
    
    print("Test 2: Advanced Job Search with Filters")
    print("Query: 'I want to find remote Senior Software Engineer positions...'")
    print("\n🤖 Agent Response:")
    
    response_3 = agent.invoke({"input": user_query_3})
    print(response_3.get("output", "No response"))
    
except Exception as e:
    print(f"❌ Exception during advanced job search: {e}")

print("\n" + "="*60)


In [ ]:
# Test job_search_from_cv_tool.py functions
try:
    from job_search_from_cv_tool import search_jobs_from_cv
    print("✅ Successfully imported job search from CV tool")
    print(f"search_jobs_from_cv function: {search_jobs_from_cv}")
except ImportError as e:
    print(f"❌ Job search from CV tool import error: {e}")

# Sample CV for testing
sample_cv_for_search = """
Alice Johnson
Data Scientist

Contact Information:
Email: alice.johnson@email.com
Phone: +1-555-987-6543
Location: New York, NY

Professional Summary:
Experienced data scientist with 4+ years in machine learning and analytics.
Specialized in Python, R, and cloud platforms.

Work Experience:
Data Scientist | DataCorp Inc. | 2020 - Present
New York, NY
- Built ML models for customer segmentation using Python and scikit-learn
- Developed data pipelines with Apache Spark and AWS
- Led analytics projects for 500k+ user base

Technical Skills:
- Programming: Python, R, SQL, JavaScript  
- ML/AI: scikit-learn, TensorFlow, PyTorch
- Cloud: AWS, GCP, Azure
- Databases: PostgreSQL, MongoDB

Education:
MS Data Science | Columbia University | 2018-2020
"""

# Test search_jobs_from_cv function
print("\n=== Testing search_jobs_from_cv Function ===")
try:
    # Test function call (will fail without API keys, but tests interface)
    result = search_jobs_from_cv.func(
        cv_content=sample_cv_for_search,
        cv_content_type="text",
        location="",  # Let it use CV location
        num_results=3,
        include_entry_level=False,
        model_name=MODEL_NAME
    )
    
    print(f"✅ Function called successfully")
    print(f"Success: {result.get('success', False)}")
    
    if result.get('success'):
        print(f"CV analysis success: {result.get('cv_analysis', {}).get('success', False)}")
        print(f"Job search success: {result.get('job_search_success', False)}")
        print(f"Jobs found: {result.get('jobs_found', 0)}")
        
        # Show strategy if available
        strategy = result.get('search_strategy', {})
        if strategy:
            print(f"Generated keyword: {strategy.get('keyword', 'N/A')}")
            print(f"Generated location: {strategy.get('location', 'N/A')}")
            print(f"Experience level: {strategy.get('experience_level', 'N/A')}")
            
    else:
        error = result.get('error', '')
        if 'api' in error.lower() or 'credential' in error.lower():
            print("✅ Expected error: Missing API credentials")
        elif 'llm' in error.lower() or 'groq' in error.lower():
            print("✅ Expected error: LLM setup required")
        else:
            print(f"Error: {error}")
        
except Exception as e:
    print(f"❌ search_jobs_from_cv test error: {e}")

# Test with different CV content types
print("\n=== Testing Different Content Types ===")
test_cases = [
    {
        "name": "PDF path test",
        "cv_content": "non_existent_resume.pdf",
        "cv_content_type": "pdf"
    },
    {
        "name": "Empty content test", 
        "cv_content": "",
        "cv_content_type": "text"
    },
    {
        "name": "Invalid content type test",
        "cv_content": "Some CV content",
        "cv_content_type": "invalid_type"
    }
]

for i, test_case in enumerate(test_cases, 1):
    print(f"\n--- Test {i}: {test_case['name']} ---")
    try:
        result = search_jobs_from_cv.func(
            cv_content=test_case["cv_content"],
            cv_content_type=test_case["cv_content_type"],
            num_results=1,
            model_name=MODEL_NAME
        )
        
        print(f"Success: {result.get('success', False)}")
        
        if not result.get('success'):
            error = result.get('error', '')
            if 'empty' in error.lower() or 'not found' in error.lower() or 'invalid' in error.lower():
                print("✅ Expected error: Proper validation")
            else:
                print(f"Error: {error[:100]}...")
        else:
            print("⚠️ Unexpected success")
            
    except Exception as e:
        print(f"❌ Test {i} error: {e}")


## 3. Test Agent Smart CV-Based Job Search (STAR FEATURE)

Testing the agent's most advanced capability: automatically finding jobs based on CV analysis

In [ ]:
print("🧪 Testing Agent Smart CV-Based Job Search (STAR FEATURE)")
print("=" * 60)

try:
    # Test: User provides CV and asks agent to find matching jobs
    user_query_4 = f"""Here is my CV. Can you analyze it and automatically find job opportunities that match my background? 
    Please show me relevant positions that fit my experience level, skills, and location preferences.

{sample_cv_text}

I'm open to remote work and would prefer senior-level positions. Please find me some good opportunities and explain why they're a good fit for my profile."""
    
    print("Test: Smart CV-Based Job Matching")
    print("Query: 'Here is my CV. Can you analyze it and automatically find job opportunities...'")
    print("\n🤖 Agent Response:")
    
    response_4 = agent.invoke({"input": user_query_4})
    print(response_4.get("output", "No response"))
    
except Exception as e:
    print(f"❌ Exception during smart CV-based job search: {e}")

print("\n" + "="*60)


## 4. Test Agent PDF Handling and Career Guidance

Testing the agent's ability to handle PDF files and provide career advice

In [ ]:
print("🧪 Testing Agent PDF Handling and Career Guidance")
print("=" * 60)

try:
    # Test 1: Career advice request
    user_query_5 = """I'm a software engineer with 7+ years of experience in Python and JavaScript. 
    I want to transition to a more senior role, possibly as a tech lead or architect. 
    What should I focus on to make this transition? And can you help me find relevant positions?"""
    
    print("Test 1: Career Guidance Request")
    print("Query: 'I want to transition to a more senior role, possibly as a tech lead...'")
    print("\n🤖 Agent Response:")
    
    response_5 = agent.invoke({"input": user_query_5})
    print(response_5.get("output", "No response"))
    
except Exception as e:
    print(f"❌ Exception during career guidance: {e}")

print("\n" + "="*60)

try:
    # Test 2: PDF file handling (conceptual test)
    user_query_6 = """If I had a PDF resume file, how would you help me analyze it and find jobs? 
    What's the process you would follow? Can you explain your capabilities for handling PDF resumes?"""
    
    print("Test 2: PDF Handling Capabilities")
    print("Query: 'If I had a PDF resume file, how would you help me analyze it...'")
    print("\n🤖 Agent Response:")
    
    response_6 = agent.invoke({"input": user_query_6})
    print(response_6.get("output", "No response"))
    
except Exception as e:
    print(f"❌ Exception during PDF capabilities inquiry: {e}")

print("\n" + "="*60)
